In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains

from webdriver_manager.chrome import ChromeDriverManager

navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [2]:
# Indicando onde está o arquivo base do Policarpo
planilha = 'vendas_saulo0624'
pvs_lista = pd.read_excel(f'{planilha}.xlsx')

# Removendo linhas duplicadas com base em uma coluna
pvs_lista = pvs_lista.drop_duplicates(subset=['PV'])
# Excluindo valores nan
pvs_lista = pvs_lista.dropna(subset=['PV'])
# Convertendo float em int e salvando uma lista com tolist()
pvs = pvs_lista['PV'].astype(int).tolist()

# pvs =[359931, 359963, 359743]
len(pvs)

42

In [4]:
navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [5]:
# navegador = webdriver.Chrome()
navegador.implicitly_wait(20) # seconds
navegador.maximize_window()
navegador.get("https://www2.lopesnet.com.br/")

# Login
navegador.find_element(By.XPATH,
                    '//*[@id="txtUsuario"]').send_keys("helder.constancio")

# Senha
navegador.find_element(By.XPATH,
                    '//*[@id="txtSenha"]').send_keys("Lopes3009!@")

navegador.find_element(By.XPATH,
                    '//*[@id="btnEntrar"]').click()

select = Select(navegador.find_element(by=By.ID, value='cboModulosAcessoUsuario'))

# select by visible text
select.select_by_visible_text('Intermediações')

# Informações externas
pvs

# Salvando informações dos clientes
dados_pv1 = {}

# Salvando informações das propostas
dados_pv2 = {}

# Percorrendo os dados cadastrais
for pv in pvs:
    
    # Informando PV para pesquisa    
    navegador.find_element(By.XPATH, '//*[@id="ctl00_conteudo_txtNumPV"]').send_keys(Keys.CONTROL, 'a')
    navegador.find_element(By.XPATH, '//*[@id="ctl00_conteudo_txtNumPV"]').send_keys(Keys.BACKSPACE)
    time.sleep(5)
    navegador.find_element(By.XPATH, '//*[@id="ctl00_conteudo_txtNumPV"]').send_keys(pv)
    time.sleep(5)
    navegador.find_element(By.XPATH, '//*[@id="ctl00_conteudo_txtNumPV"]').send_keys(Keys.ENTER)
    time.sleep(5)

     # Verificar status da pv
    status_pv = navegador.find_element(By.XPATH, '//*[@id="ctl00_conteudo_conteudoPesquisaPV"]/tr/td[1]')
    status_pv = status_pv.text
    status_pv = status_pv.replace(f'{str(pv)}\n', '')


    if status_pv == 'Pré-Escriturado' or status_pv == 'Escrituração':

        resumo_pv = navegador.find_element(by=By.TAG_NAME, value='tbody')

        acesso_pv = resumo_pv.find_element(by=By.CLASS_NAME, value='destaque')
        acesso_pv.click()

        # Informações do pv
        status_pv = navegador.find_element(by=By.CLASS_NAME, value='secao-status')
        status_pv = status_pv.find_element(by=By.TAG_NAME, value='span').text
        time.sleep(5)
        # Navegando na parte do cliente
        cliente = navegador.find_elements(by=By.NAME, value='divComprador')
        cliente1 = navegador.find_elements(by=By.NAME, value='divCoparticipante')

        # Salvando os links para acesso ao cadastro dos clientes
        # divComprador
        csv_cliente = []
        for c in cliente:
            csv = c.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
            csv_cliente.append(csv)

        trava = 'Empresa Franqueador I\nLPS Soluções Imobiliarias e Participações LTDA\nCPF/CNPJ: 20.912.827/0001-89'
        for i in cliente1:
            if i.text != trava:
                c = i.find_element(by=By.TAG_NAME, value='a').get_attribute('onclick')
                c = c.replace('javascript:carregarTelaEditarPessoa(', '')
                c = c.replace(');', '')
                a = f'https://vendas.lopesnet.com.br/Vendas/AtualizarPessoa/APDadosGerais.aspx?c={c}&v={pv}'
                csv_cliente.append(a)
            else:
                break


        # Coletando as informações dos clientes
        dados = {}
        contador = 0

        for cliente in csv_cliente:
            info_cliente = cliente
            navegador.get(info_cliente)
            dados1 = navegador.find_elements(by=By.CLASS_NAME, value='vendas-grid-1-5')
            dados2 = navegador.find_elements(by=By.CLASS_NAME, value='vendas-grid-2-5')
            cpfcnpj = dados1[0].find_element(by=By.TAG_NAME, value='input').get_attribute('value')

            if len(cpfcnpj) > 15:
                nome = dados2[0].find_element(by=By.TAG_NAME, value='input').get_attribute('value')

                email = dados2[4].find_element(by=By.TAG_NAME, value='input').get_attribute('value')

                # Pegando o url da página atual
                url = navegador.current_url
                ciav = url.replace('https://vendas.lopesnet.com.br/Vendas/AtualizarPessoa/APDadosGerais.aspx?c=', '')
                ciav = ciav.replace(f'&v={pv}', '')
                
                try:
                    endereco = navegador.get(f'https://vendas.lopesnet.com.br/Vendas/AtualizarPessoa/APEnderecos.aspx?c={ciav}&v={pv}')
                    endereco = navegador.find_element(by=By.CLASS_NAME, value='vendas-item-resumo').text
                    navegador.find_element(by=By.CLASS_NAME, value='vendas-item-resumo').click()
                    time.sleep(3)
                    div_endereco = navegador.find_element(By.CLASS_NAME, 'vendas-item').get_attribute('id')
                    div_endereco = div_endereco.replace('div_', '')
                    cep = navegador.find_element(By.ID, f'txtCepAp_{div_endereco}').get_attribute('value')
                    bairro = navegador.find_element(By.ID, f'txtBairroAp_{div_endereco}').get_attribute('value')
                    select = Select(navegador.find_element(By.ID, f'ddlEstadoAp_{div_endereco}'))
                    uf = select.first_selected_option.text
                    sele = Select(navegador.find_element(By.ID, f'ddlCidadeAp_{div_endereco}'))
                    cidade = sele.first_selected_option.text
                except NoSuchElementException as erro:
                    endereco = 'N/D'
                
                try:
                    telefone = navegador.get(f'https://vendas.lopesnet.com.br/Vendas/AtualizarPessoa/APTelefones.aspx?c={ciav}&v={pv}')
                    telefone = navegador.find_element(by=By.CLASS_NAME, value='vendas-item-resumo').text
                except NoSuchElementException as erro:
                    telefone = 'N/D'

                dados_clientes = {
                    'CPF/CNPJ': cpfcnpj,
                    'Nome': nome,
                    'E-mail': email,
                    'Endereço': endereco,
                    'Cep': cep,
                    'Bairro': bairro,
                    'UF': uf,
                    'Cidade': cidade,
                    'Telefone': telefone
                }

                d = f'Clientepj{contador}'
                dados[d] = dados_clientes

                contador = contador + 1

            else:
                nome = dados2[0].find_element(by=By.TAG_NAME, value='input').get_attribute('value')

                select = Select(navegador.find_element(by=By.ID, value='ctl00_conteudo_UcAPDadosGerais_ddlSexo'))
                sexo = select.first_selected_option.text

                data_nasci = dados1[2].find_element(by=By.TAG_NAME, value='input').get_attribute('value')

                select = Select(navegador.find_element(by=By.ID, value='ctl00_conteudo_UcAPDadosGerais_ddlNacionalidade'))
                nacionalidade = select.first_selected_option.text

                select = Select(navegador.find_element(by=By.ID, value='ctl00_conteudo_UcAPDadosGerais_ddlEstadoNacionalidade'))
                estado = select.first_selected_option.text

                naturalidade = dados2[4].find_element(by=By.TAG_NAME, value='input').get_attribute('value')

                email = dados2[5].find_element(by=By.TAG_NAME, value='input').get_attribute('value')

                profissao = dados2[8].find_element(by=By.TAG_NAME, value='input').get_attribute('value')

                select = Select(navegador.find_element(by=By.ID, value='ctl00_conteudo_UcAPDadosGerais_ddlEstadoCivil'))
                estado_civil = select.first_selected_option.text

                # Pegando o url da página atual
                url = navegador.current_url
                ciav = url.replace('https://vendas.lopesnet.com.br/Vendas/AtualizarPessoa/APDadosGerais.aspx?c=', '')
                ciav = ciav.replace(f'&v={pv}', '')

                try:
                    endereco = navegador.get(f'https://vendas.lopesnet.com.br/Vendas/AtualizarPessoa/APEnderecos.aspx?c={ciav}&v={pv}')
                    endereco = navegador.find_element(by=By.CLASS_NAME, value='vendas-item-resumo').text
                    navegador.find_element(by=By.CLASS_NAME, value='vendas-item-resumo').click()
                    time.sleep(3)
                    div_endereco = navegador.find_element(By.CLASS_NAME, 'vendas-item').get_attribute('id')
                    div_endereco = div_endereco.replace('div_', '')
                    cep = navegador.find_element(By.ID, f'txtCepAp_{div_endereco}').get_attribute('value')
                    bairro = navegador.find_element(By.ID, f'txtBairroAp_{div_endereco}').get_attribute('value')
                    select = Select(navegador.find_element(By.ID, f'ddlEstadoAp_{div_endereco}'))
                    uf = select.first_selected_option.text
                    sele = Select(navegador.find_element(By.ID, f'ddlCidadeAp_{div_endereco}'))
                    cidade = sele.first_selected_option.text
                except NoSuchElementException as erro:
                    endereco = 'N/D'

                try:
                    telefone = navegador.get(f'https://vendas.lopesnet.com.br/Vendas/AtualizarPessoa/APTelefones.aspx?c={ciav}&v={pv}')
                    telefone = navegador.find_element(by=By.CLASS_NAME, value='vendas-item-resumo').text
                except NoSuchElementException as erro:
                    telefone = 'N/D'

                dados_clientes = {
                    'CPF/CNPJ': cpfcnpj,
                    'Nome': nome,
                    'Sexo': sexo,
                    'Data de Nascimento': data_nasci,
                    'Nacionalidade': nacionalidade,
                    'Estado': estado,
                    'Naturalidade': naturalidade,
                    'E-mail': email,
                    'Profissão': profissao,
                    'Estado Civil': estado_civil,
                    'Endereço': endereco,
                    'CEP': cep,
                    'Bairro': bairro,
                    'UF': uf,
                    'Cidade': cidade,
                    'Telefone': telefone
                }
                
                

                d = f'Clientepf{contador}'
                dados[d] = dados_clientes
                
                dados_pv_teste = {pv:{status_pv:dados}}
                
                contador = contador + 1
            
#             dados_pv1.update(dados)


        
        dados_pv1.update(dados_pv_teste)
        

    # Coletando o fluxo da proposta
    navegador.get(f'https://vendas.lopesnet.com.br/Vendas/Proposta.aspx?c={pv}')
    proposta = navegador.find_element(by=By.CLASS_NAME, value='tabela-interna-compacta')
    tabela = proposta.find_element(by=By.TAG_NAME, value='tbody')
    linhas = tabela.find_elements(by=By.TAG_NAME, value='tr')

    contador = 0
    proposta = {}
    for linha in linhas:
        proposta[contador] = linha.text
        contador += 1

    
    dados_pv_p = {pv:proposta}
    dados_pv2.update(dados_pv_p)
    
    navegador.get('https://vendas.lopesnet.com.br/Vendas/PesquisaPV.aspx')

time.sleep(15)
navegador.close()

In [6]:
import openpyxl
wb = openpyxl.load_workbook('Relatório_Clientes.xlsx')
sheet = wb['dados']


linha = 2
for pv in dados_pv1.keys():
    for status in dados_pv1[pv]:
        for cliente in dados_pv1[pv][status]:
            for dados in list(dados_pv1[pv][status][cliente].values()):
                sheet.cell(row=linha, column=1).value = pv
                sheet.cell(row=linha, column=2).value = status
                sheet.cell(row=linha, column=3).value = cliente
                sheet.cell(row=linha, column=4).value = dados
                linha += 1

wb.save('Relatório_Clientes_0624.xlsx')

In [7]:
wb = openpyxl.load_workbook('Relatório_Fluxo.xlsx')
sheet = wb['dados']

linha = 2
for pv in dados_pv2.keys():
    for parcela, fluxo in zip(dados_pv2[pv], dados_pv2[pv].values()):
        sheet.cell(row=linha, column=1).value = pv
        sheet.cell(row=linha, column=2).value = parcela
        sheet.cell(row=linha, column=3).value = fluxo
        linha += 1

wb.save('Relatório_Fluxo_0624.xlsx')